In [19]:
##Test A: Cientifico de Datos - Prueba Reconocimiento de Genero
##Por: Samantha Valdez

##Construir el contador de reconocimiento facial no es sencillo, por ello me base en un enfoque de Arquitectura modelo CNN y 
##Cascada en el que las imagenes pasan por un clasificador, luego las partes que nos interesan de la imagen son extraidas 
##de esta, creando muestras procedemos a modificar el tamano de la imagen para un mejor analisis y esa imagen modificada 
##la usamos para evaluar un modelo CNN entrenado

##A continuacion importo las librerias que necesito (Tensorflow,Keras, h5py, numpy y OpenCV de python) 
##para crear un modelo usando una Red Neuronal Convolucional

import numpy as np
import os
import cv2
import random
import glob
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, Conv2D, Dense
from tensorflow.keras.layers import MaxPooling2D, Activation, Flatten, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical, plot_model

In [20]:
##Para el procesamiento de la data, necesitare cambiar el tamaño de acuerdo a las especificaciones del modelo que diseñare

##Defino los parametros iniciales
lr = 1e-2 #es un parametro de ajuste que determina el tamaño del paso en cada iteración
batch_size = 32 #numero de ejemplos de entrenamiento pasados en una iteración
epochs = 100 #el ciclo a traves del conjunto de datos de entrenamiento completo
img_dims = (96,96,3) #especifico las dimensiones de la imagen

#lists used to store images and its corresponding labels
data = [] 
labels = []

#cargo los archivos
image_files = [f for f in glob.glob(r'C:\Users\Samantha Valdez\Desktop\PythonProjects\genero' + "/**/*",
recursive=True) if not os.path.isdir(f)] #almaceno todas las imagenes segun la ruta

random.shuffle(image_files)

In [21]:
##Conversion de imagenes a un array  
##Como en el procesamiento se espera que la data este en un formato numerico, asi podre poner las imagenes en un array usando 
##la libreria de Keras
for img in image_files:

    image = cv2.imread(img)
    image = cv2.resize(image, (img_dims[0],img_dims[1])) #Cargando cada imagen y cambiando el tamaño de acuerdo con 
                                                         #las dimensiones que quiero

    image = img_to_array(image) #Convierto las imagenes redimensionadas en un array
    data.append(image) #agrego las imagenes a una lista de datos

    ##Identifico las categorias (Man=0 y Woman=1)
    label = img.split(os.path.sep)[-2] 
    if label == "woman":
        label = 1
    else:
        label = 0
        
    labels.append([label])

In [22]:
##Ahora normalizare los datos y convirtiendo las etiquetas en un array.
##Pre-procesado
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

##Separare la data en porcentajes (uno de entrenamiento y otro de validacion)
##Divido el conjunto de datos para el entrenamiento y validacion en una proporción de 80 % de datos de entrenamiento y 20 % de datos de prueba
x_train,x_test,y_train,y_test = train_test_split(data, labels,
test_size=0.2,random_state=42)

##Convierto en etiquetas categoricas 
y_train = to_categorical(y_train, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)

In [23]:
#aumento el conjunto de datos
##este proceso de aumentar la cantidad y diversidad de datos se utiliza para aumentar la cantidad de datos 
##ayuda a reducir el sobreajuste al entrenar un modelo
aug = ImageDataGenerator(rotation_range=25,   #rango de grados para rotaciones aleatorias
                         width_shift_range=0.1, #fraccion del ancho total
                         height_shift_range=0.1, #fraccion de la altura total
                         shear_range=0.2,        #angulo de corte en sentido antihorario en grados
                         zoom_range=0.2,         #rango para zoom aleatorio
                         horizontal_flip=True,   #voltear aleatoriamente las entradas horizontalmente 
                         fill_mode="nearest")    #rellenar Los puntos fuera de los límites de la entrada de acuerdo con 
                                                 #el modo dado en las imágenes

In [24]:
##Defino el modelo convolucional
#defino la forma de entrada
width = img_dims[0]
height = img_dims[1]
depth = img_dims[2]
inputShape = (height, width, depth) #defino la forma de entrada y sus dimensiones
dim = -1

#creo el modelo
model = Sequential() #creo un modelo secuencia con 5 capas ocultas conv2D con la secuencia de 32,64,64,128 y 256 neuronas ocultas 
                     #(usando relu activation function, batchnormalization,maxpooling2D y Dropout)

model.add(Conv2D(32, (3,3), padding="same", input_shape=inputShape))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=dim))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3,3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=dim))

model.add(Conv2D(64, (3,3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=dim))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3,3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=dim))

model.add(Conv2D(256, (3,3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=dim))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten()) #Aplano los datos (los convierto en un array unidimensional para ingresarlos en la siguiente capa)

#Capa densa con 1024 neuronas, funcion de activación relu, normalización por lotes y tasa de abandono del 50%
model.add(Dense(1024)) 
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))

#Capa de salida final con 2 neuronas y funcion de activación sigmoid
model.add(Dense(2))
model.add(Activation("sigmoid"))

In [25]:
##Compilo el modelo
#Usando el algoritmo de Adam Optimizer
opt = tf.keras.optimizers.Adam(lr=lr)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

##Ajusto el modelo
h = model.fit_generator(aug.flow(x_train, y_train, batch_size=batch_size),
                        validation_data=(x_test,y_test),
                        steps_per_epoch=len(x_train) // batch_size,
                        epochs=epochs, verbose=1)

##guardo el modelo
model.save('gender_predictor.model')

Epoch 1/100


C:\Users\Samantha Valdez\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
C:\Users\Samantha Valdez\AppData\Local\Temp\ipykernel_20244\1038399749.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  h = model.fit_generator(aug.flow(x_train, y_train, batch_size=batch_size),


40/40 [==============================] - 14s 335ms/step - loss: 1.1480 - accuracy: 0.5906 - val_loss: 17.7847 - val_accuracy: 0.6094
Epoch 2/100
40/40 [==============================] - 13s 327ms/step - loss: 1.0584 - accuracy: 0.6109 - val_loss: 21.1604 - val_accuracy: 0.5500
Epoch 3/100
40/40 [==============================] - 13s 330ms/step - loss: 0.8480 - accuracy: 0.6570 - val_loss: 8.8204 - val_accuracy: 0.4906
Epoch 4/100
40/40 [==============================] - 13s 324ms/step - loss: 0.7525 - accuracy: 0.7102 - val_loss: 3.6250 - val_accuracy: 0.5656
Epoch 5/100
40/40 [==============================] - 13s 324ms/step - loss: 0.6646 - accuracy: 0.7258 - val_loss: 2.9855 - val_accuracy: 0.6438
Epoch 6/100
40/40 [==============================] - 13s 322ms/step - loss: 0.5890 - accuracy: 0.7523 - val_loss: 0.6506 - val_accuracy: 0.7281
Epoch 7/100
40/40 [==============================] - 13s 323ms/step - loss: 0.5766 - accuracy: 0.7820 - val_loss: 0.8600 - val_accuracy: 0.6562
Ep

40/40 [==============================] - 13s 319ms/step - loss: 0.2373 - accuracy: 0.8914 - val_loss: 0.2568 - val_accuracy: 0.8906
Epoch 58/100
40/40 [==============================] - 13s 323ms/step - loss: 0.2424 - accuracy: 0.8961 - val_loss: 0.2387 - val_accuracy: 0.9094
Epoch 59/100
40/40 [==============================] - 13s 322ms/step - loss: 0.2228 - accuracy: 0.9055 - val_loss: 0.1428 - val_accuracy: 0.9406
Epoch 60/100
40/40 [==============================] - 13s 322ms/step - loss: 0.1811 - accuracy: 0.9273 - val_loss: 0.2382 - val_accuracy: 0.9062
Epoch 61/100
40/40 [==============================] - 13s 322ms/step - loss: 0.2028 - accuracy: 0.9297 - val_loss: 0.1743 - val_accuracy: 0.9219
Epoch 62/100
40/40 [==============================] - 13s 322ms/step - loss: 0.1744 - accuracy: 0.9297 - val_loss: 0.1246 - val_accuracy: 0.9438
Epoch 63/100
40/40 [==============================] - 13s 324ms/step - loss: 0.1947 - accuracy: 0.9203 - val_loss: 0.1194 - val_accuracy: 0.956

INFO:tensorflow:Assets written to: gender_predictor.model\assets


INFO:tensorflow:Assets written to: gender_predictor.model\assets


In [42]:
##Cargo el modelo de Prediccion de genero
#importo las librerias que necesito 
import cv2
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np


#cargo el modelo
model = load_model("gender_predictor.model")

cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

#defino las clases
classes = ['men','women']

In [43]:
def gender_facecounter(image, m, f, size=0.5): #tomo la imagen
    #convierto la imagen en imagen en escala de grises y la guardo en una variable llamada 'gray_scale'
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    faces = cascade.detectMultiScale(gray_image, 1.1,5)  #utilizo el metodo 'detectMultiScale' para detectar objetos de tamaños diferentes en la imagen
                                                         #aqui defino que los objetos detectados sean devueltos como una lista de rectangulos y que se 
                                                         #guarden en la variable 'faces'
    #iterando sobre caras

    for (x,y,w,h) in faces:
        cv2.rectangle(image, (x,y), (x+w,y+h), (0,255,0),3) #dibujo un rectangulo de la imagen usando 'cv2.rectangle'

        #recorto la imagen, normalizandola y llevando a un array, luego expandiendo su dimension. La guardo en una variable 'res_face'
        
        cropped_image = np.copy(image[y:y+h,x:x+w]) 

        ##preproceso las imagenes segun este modelo
        res_face = cv2.resize(cropped_image, (96,96))
        ## cv2.imshow("cropped image",res_face)
        res_face = res_face.astype("float") / 255.0
        res_face = img_to_array(res_face)
        res_face = np.expand_dims(res_face, axis=0)

        ##prediccion del modelo
        #paso el resultado de la variable 'res_face' al modelo entrenado y guardo el resultado en la variable 'result'
        
        result = model.predict(res_face)[0]

        ##obtengo la etiqueta con la máxima precisión
        idx = np.argmax(result)  #uso 'argmax' para encontrar la clase con la mayor probabilidad predicha por el resultado
        label = classes[idx]

        ##calculando el conteo 
        #si el 'label' es igual a 'woman', incremento la variable 'f' sino la variable 'm'
        
        if label == "women":
            f = f+1
        else:
            m = m+1

    cv2.rectangle(image,(0,0),(300,30),(255,255,255),-1)
    cv2.putText(image, " females = {},males = {} ".format(f,m),(0,15),
    cv2.FONT_HERSHEY_TRIPLEX,0.6,(255, 101, 125),1)
    cv2.putText(image, " faces detected = " + str(len(faces)),(10,30), #uso este metodo para dibujar un string de la imagen
    cv2.FONT_HERSHEY_TRIPLEX,0.5,(0,0,0),1)

    return image #muestro los resultados de la imagen

In [44]:
##Pruebo con una imagen
image = cv2.imread("C:\\Users\\Samantha Valdez\\Desktop\\PythonProjects\\genero\\woman\\face_1028.jpg") #leo la imagen especificando el nombre en la ruta

##mantenimiento de contadores separados para los dos generos
males = 0
females = 0

##ver resultado en ventana emergente
cv2.imshow("Gender FaceCounter", gender_facecounter(image,males,females)) #paso la imagen de entrada junto con las variables del contador
cv2.waitKey(0)
cv2.destroyAllWindows() #finalmente cierro todas las ventanas o pestanas activas

1/1 [==============================] - 0s 144ms/step
